# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import pandas_datareader.data as pdr
import yfinance as yf
import matplotlib
from scipy import stats
import yahoo_fin.stock_info as si

yf.pdr_override()

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks=pd.read_csv("constituents.csv")
for i in range(0,len(stocks["Symbol"])):
    if "." in stocks["Symbol"][i]:
        stocks["Symbol"][i]=stocks["Symbol"][i].replace(".","-")
stocks

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott,Health Care
3,ABBV,AbbVie,Health Care
4,ACN,Accenture,Information Technology
...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary
499,ZBRA,Zebra Technologies,Information Technology
500,ZBH,Zimmer Biomet,Health Care
501,ZION,Zions Bancorporation,Financials


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [4]:
symbol="AAPL"
data=pdr.get_quote_yahoo(symbol)
# data["priceEpsCurrentYear"]
data.columns

Index(['language', 'region', 'quoteType', 'typeDisp', 'quoteSourceName',
       'triggerable', 'customPriceAlertConfidence', 'currency', 'marketState',
       'regularMarketChangePercent', 'regularMarketPrice', 'exchange',
       'shortName', 'longName', 'messageBoardId', 'exchangeTimezoneName',
       'exchangeTimezoneShortName', 'gmtOffSetMilliseconds', 'market',
       'esgPopulated', 'firstTradeDateMilliseconds', 'priceHint',
       'preMarketChange', 'preMarketChangePercent', 'preMarketTime',
       'preMarketPrice', 'regularMarketChange', 'regularMarketTime',
       'regularMarketDayHigh', 'regularMarketDayRange', 'regularMarketDayLow',
       'regularMarketVolume', 'regularMarketPreviousClose', 'bid', 'ask',
       'bidSize', 'askSize', 'fullExchangeName', 'financialCurrency',
       'regularMarketOpen', 'averageDailyVolume3Month',
       'averageDailyVolume10Day', 'fiftyTwoWeekLowChange',
       'fiftyTwoWeekLowChangePercent', 'fiftyTwoWeekRange',
       'fiftyTwoWeekHighChange

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [5]:
price=data["price"]
pe_ratio=data["priceEpsCurrentYear"]
pe_ratio

AAPL    21.841166
Name: priceEpsCurrentYear, dtype: float64

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Symbol'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_df=pd.DataFrame(columns=my_columns)
for symbol_group in symbol_groups:
    data=pdr.get_quote_yahoo(symbol_group)[["price","priceEpsCurrentYear"]]
    data["nan"]=np.nan
    data.reset_index(inplace=True)
    data=data.set_axis(my_columns,axis=1)
    final_df=pd.concat([final_df,data])
final_df.reset_index(inplace=True,drop=True)


In [11]:
final_df_tmp=final_df
final_df_tmp

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
29,AAL,17.02,567.33340,NaN
489,WDC,37.22,531.71430,NaN
458,UDR,40.15,174.56522,NaN
487,WELL,71.15,169.40477,NaN
144,DXCM,108.47,135.58750,NaN
...,...,...,...,...
467,VTR,48.99,-4899.00050,NaN
169,EMR,97.92,NaN,NaN
236,HPQ,27.91,NaN,NaN
287,L,60.22,NaN,NaN


In [21]:
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
29,AAL,17.02,567.33340,NaN
489,WDC,37.22,531.71430,NaN
458,UDR,40.15,174.56522,NaN
487,WELL,71.15,169.40477,NaN
144,DXCM,108.47,135.58750,NaN
...,...,...,...,...
467,VTR,48.99,-4899.00050,NaN
169,EMR,97.92,NaN,NaN
236,HPQ,27.91,NaN,NaN
287,L,60.22,NaN,NaN


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [22]:
final_df.sort_values("Price-to-Earnings Ratio",ascending=False, inplace=True)
final_df=final_df[final_df["Price-to-Earnings Ratio"]>0]
final_df=final_df[:50]
final_df.reset_index(inplace=True,drop=True)

In [23]:
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,AAL,17.02,567.333400,NaN
1,WDC,37.22,531.714300,NaN
2,UDR,40.15,174.565220,NaN
3,WELL,71.15,169.404770,NaN
4,DXCM,108.47,135.587500,NaN
5,LYV,74.66,87.835300,NaN
6,ILMN,201.11,85.944450,NaN
7,EQIX,721.89,83.455500,NaN
8,CDAY,67.62,78.627910,NaN
9,DLR,106.00,76.258995,NaN


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [24]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [25]:
portfolio_input()

That's not a number! 
 Try again:


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [29]:
position_size=float(portfolio_size)/len(final_df.index)
# position_size
for row in final_df.index:
    final_df.loc[row,"Number of Shares to Buy"]=math.floor(position_size/final_df.loc[row,"Price"])
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,AAL,17.02,567.333400,11750.0
1,WDC,37.22,531.714300,5373.0
2,UDR,40.15,174.565220,4981.0
3,WELL,71.15,169.404770,2810.0
4,DXCM,108.47,135.587500,1843.0
5,LYV,74.66,87.835300,2678.0
6,ILMN,201.11,85.944450,994.0
7,EQIX,721.89,83.455500,277.0
8,CDAY,67.62,78.627910,2957.0
9,DLR,106.00,76.258995,1886.0


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [27]:
def strtoint(x):
    m = {'K': 3, 'M': 6, 'B': 9, 'T': 12}
    return int(float(x[:-1]) * 10 ** m[x[-1]] / 1000)
symbol="AAPL"
data=pdr.get_quote_yahoo(symbol)[["price","priceEpsCurrentYear",'priceToBook']]
dataval=si.get_stats_valuation(symbol)
dataval.set_index(0,inplace=True)
data["EV/EBITDA"]=float(dataval.loc["Enterprise Value/EBITDA"])
data["PS"]=float(dataval.loc["Price/Sales (ttm)"])
dataval2=si.get_stats(symbol)
dataval2.set_index("Attribute",inplace=True)
data["EV/GP"]=strtoint(dataval2.loc["EBITDA"][0])/strtoint(dataval2.loc["Gross Profit (ttm)"][0])
data

/home/hakoshie/.local/lib/python3.8/site-packages/yahoo_fin/stock_info.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(elt)
/home/hakoshie/.local/lib/python3.8/site-packages/yahoo_fin/stock_info.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(elt)
/home/hakoshie/.local/lib/python3.8/site-packages/yahoo_fin/stock_info.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(elt)
/home/hakoshie/.local/lib/python3.8/site-packages/yahoo_fin/stock_info.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(elt)
/home/hakoshie/.local/lib/python3.8/site

,price,priceEpsCurrentYear,EV/EBITDA,PS,EV/GP
AAPL,134.76,21.841166,NaN,NaN,0.764375


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [30]:
symbol="AAPL"
dataval2=si.get_stats_valuation(symbol)
dataval2.set_index(0,inplace=True)
dataval2

,1
0,
Market Cap (intraday),NaN
Enterprise Value,NaN
Trailing P/E,NaN
Forward P/E,NaN
PEG Ratio (5 yr expected),NaN
Price/Sales (ttm),NaN
Price/Book (mrq),NaN
Enterprise Value/Revenue,NaN
Enterprise Value/EBITDA,NaN


In [28]:
symbol="MSFT"
dataval2=si.get_stats(symbol)
dataval2.set_index("Attribute",inplace=True)
dataval2

/home/hakoshie/.local/lib/python3.8/site-packages/yahoo_fin/stock_info.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(elt)
/home/hakoshie/.local/lib/python3.8/site-packages/yahoo_fin/stock_info.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(elt)
/home/hakoshie/.local/lib/python3.8/site-packages/yahoo_fin/stock_info.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(elt)
/home/hakoshie/.local/lib/python3.8/site-packages/yahoo_fin/stock_info.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(elt)
/home/hakoshie/.local/lib/python3.8/site

,Value
Attribute,
Beta (5Y Monthly),0.93
52-Week Change 3,-20.95%
S&P500 52-Week Change 3,-12.63%
52 Week High 3,315.95
52 Week Low 3,213.43
50-Day Moving Average 3,239.97
200-Day Moving Average 3,257.76
Avg Vol (3 month) 3,30.23M
Avg Vol (10 day) 3,31.31M


In [34]:
rv_columns=[
    "Ticker",
    "Price",
    "Number of Shares to Buy",
    "Price-to-Earnings Ratio",
    "PE Percentile",
    "Price-to-Book Ratio",
    "PB Percentile",
    "Price-to-Sales Ratio",
    "PS Percentile",
    "EV/EBITDA",
    "EV/EBITDA Percentile",
    "EV/GP",
    "EV/GP Percentile",
    "RV Score"
]
rv_df=pd.DataFrame(columns=rv_columns)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [44]:
data=pdr.get_quote_yahoo("META")[["price","priceEpsCurrentYear",'priceToBook']]
data["nan"]=data["nan2"]=np.nan
data.reindex(["price","priceEpsCurrentYear","nan",'priceToBook',"nan2"],axis=1)

,price,priceEpsCurrentYear,nan,priceToBook,nan2
META,136.98,15.085903,NaN,2.941757,NaN


In [174]:
m = {'K': 3, 'M': 6, 'B': 9, 'T': 12,'k':3}
def strtoint(x):
        val=0
        try:
                val=float(x)
        except:
                try: 
                        val=int(float(x[:-1]) * 10 ** m[x[-1]] / 1000)
                except:
                        return np.nan


        return val

In [164]:
import warnings
warnings.simplefilter('ignore', FutureWarning)
m = {'K': 3, 'M': 6, 'B': 9, 'T': 12,"k":3}
def strtoint(x):
        val=0
        try:
                val=float(x)
                # return float(x)
        except:
                print("unko")
                val=int(float(x[:-1]) * 10 ** m[x[-1]] / 1000)
        return val
for symbol in stocks["Symbol"]:
    # symbol="AAPL"
        data=pdr.get_quote_yahoo(symbol)[["price","priceEpsCurrentYear",'priceToBook']]
        data["nan0"]=data["nan1"]=data["nan2"]=np.nan
        data.reindex(["price","nan0","priceEpsCurrentYear","nan1",'priceToBook',"nan2"],axis=1) 
        dataval=si.get_stats_valuation(symbol)
        dataval.set_index(0,inplace=True)
        data["PS"]=float(dataval.loc["Price/Sales (ttm)"])
        data["nan3"]=np.nan
        data["EV/EBITDA"]=float(dataval.loc["Enterprise Value/EBITDA"])
        data["nan4"]=np.nan
        dataval2=si.get_stats(symbol)
        dataval2.set_index("Attribute",inplace=True)

# dataval["EBITDA"]
        data["EV/GP"]=strtoint(dataval2.loc["EBITDA"][0])/strtoint(dataval2.loc["Gross Profit (ttm)"][0])
        data["nan5"]=np.nan
        data["nan6"]=np.nan
        data.reset_index(inplace=True)
        data=data.set_axis(rv_columns,axis=1)
        rv_df=pd.concat([rv_df,data])
    
    

unko
unko


KeyboardInterrupt: 

In [149]:
strtoint('4.6B')

4600000.0

In [114]:
rv_df=rv_df.drop_duplicates(subset=["Ticker"])
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,129.51,12.709519,5.084207,NaN,NaN,NaN,2.14,NaN,8.71,NaN,0.420386,NaN,NaN
0,AOS,62.13,20.172080,5.430944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.546412,NaN,NaN
0,ABT,113.51,21.745213,5.547627,NaN,NaN,NaN,4.47,NaN,15.50,NaN,0.547381,NaN,NaN
0,ABBV,153.60,11.098267,16.983637,NaN,NaN,NaN,4.72,NaN,13.01,NaN,0.766461,NaN,NaN
0,ACN,282.14,24.598083,7.729864,NaN,NaN,NaN,2.90,NaN,16.97,NaN,0.537056,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,ILMN,201.11,85.944450,4.690831,NaN,NaN,NaN,6.72,NaN,-9.02,NaN,0.176471,NaN,NaN
0,INCY,80.83,29.286232,4.254211,NaN,NaN,NaN,4.71,NaN,18.63,NaN,0.506328,NaN,NaN
0,IR,57.53,25.343613,2.662070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.597990,NaN,NaN
0,INTC,30.11,15.441026,1.244061,NaN,NaN,NaN,1.78,NaN,5.22,NaN,0.475919,NaN,NaN


In [178]:
for symbol in stocks["Symbol"][250:260]:
        try:
            data=pdr.get_quote_yahoo(symbol)[["price","priceEpsCurrentYear",'priceToBook']]
        except:
            continue
        data["nan0"]=data["nan1"]=data["nan2"]=np.nan
        data.reindex(["price","nan0","priceEpsCurrentYear","nan1",'priceToBook',"nan2"],axis=1) 
        try: 
            dataval=si.get_stats_valuation(symbol)
        except:
            continue
        dataval.set_index(0,inplace=True)
        data["PS"]=strtoint(dataval.loc["Price/Sales (ttm)"])
        data["nan3"]=np.nan

        data["EV/EBITDA"]=strtoint(dataval.loc["Enterprise Value/EBITDA"])
        data["nan4"]=np.nan
        dataval2=si.get_stats(symbol)
        dataval2.set_index("Attribute",inplace=True)

# dataval["EBITDA"]
        data["EV/GP"]=strtoint(dataval2.loc["EBITDA"])/strtoint(dataval2.loc["Gross Profit (ttm)"])
        data["nan5"]=np.nan
        data["nan6"]=np.nan
        data.reset_index(inplace=True)
        data=data.set_axis(rv_columns,axis=1)
        rv_df=pd.concat([rv_df,data])

In [142]:
strtoint("4.6B")

In [179]:
rv_df=rv_df.drop_duplicates(subset=["Ticker"])
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,129.51,12.709519,5.084207,NaN,NaN,NaN,2.14,NaN,8.71,NaN,0.420386,NaN,NaN
0,AOS,62.13,20.172080,5.430944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.546412,NaN,NaN
0,ABT,113.51,21.745213,5.547627,NaN,NaN,NaN,4.47,NaN,15.50,NaN,0.547381,NaN,NaN
0,ABBV,153.60,11.098267,16.983637,NaN,NaN,NaN,4.72,NaN,13.01,NaN,0.766461,NaN,NaN
0,ACN,282.14,24.598083,7.729864,NaN,NaN,NaN,2.90,NaN,16.97,NaN,0.537056,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,IQV,222.14,21.885715,7.719091,NaN,NaN,NaN,2.98,NaN,18.27,NaN,NaN,NaN,NaN
0,IRM,52.63,29.734465,28.901703,NaN,NaN,NaN,3.09,NaN,15.80,NaN,NaN,NaN,NaN
0,JBHT,176.80,18.133333,5.222426,NaN,NaN,NaN,1.27,NaN,9.80,NaN,NaN,NaN,NaN
0,JKHY,181.76,36.719193,9.069859,NaN,NaN,NaN,6.71,NaN,20.38,NaN,NaN,NaN,NaN


In [181]:
rv_df.to_csv("to260.csv")

In [180]:
stocks["Symbol"][259]

'J'

In [75]:
for i,symbol in enumerate(stocks["Symbol"]):
    if symbol=="CSCO":
        print(i)

111


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
17,AFL,39.060,N/A,9.7,N/A,0.950934,N/A,NaN,N/A,NaN,N/A,N/A
18,AIG,31.950,N/A,-5.88,N/A,0.407454,N/A,NaN,N/A,NaN,N/A,N/A
20,AIZ,127.680,N/A,20.22,N/A,1.360264,N/A,NaN,N/A,NaN,N/A,N/A
26,ALL,97.880,N/A,7.1,N/A,1.173772,N/A,NaN,N/A,NaN,N/A,N/A
39,ANTM,293.560,N/A,12.24,N/A,2.304037,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,196.350,N/A,26.09,N/A,13.486066,N/A,17.533688,N/A,NaN,N/A,N/A
56,BAC,26.970,N/A,12.63,N/A,0.867662,N/A,NaN,N/A,NaN,N/A,N/A
64,BK,38.160,N/A,8.27,N/A,0.804550,N/A,NaN,N/A,NaN,N/A,N/A
65,BKNG,1866.170,N/A,30.78,N/A,12.571484,N/A,12.859665,N/A,NaN,N/A,N/A
75,C,55.140,N/A,9.52,N/A,0.576968,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

0      0.841584
1      0.112871
2      0.623762
3      0.740594
4      0.427723
         ...   
500         0.6
501    0.994059
502    0.693069
503    0.257426
504    0.843564
Name: PE Percentile, Length: 505, dtype: object
0       0.752475
1      0.0158416
2       0.510891
3       0.940594
4      0.0257426
         ...    
500     0.049505
501     0.415842
502     0.811881
503     0.132673
504     0.956436
Name: PB Percentile, Length: 505, dtype: object
0       0.877228
1      0.0732673
2        0.50099
3       0.861386
4       0.350495
         ...    
500     0.744554
501     0.572277
502     0.762376
503      0.69505
504     0.924752
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.552475
1      0.0574257
2      0.0653465
3       0.865347
4       0.340594
         ...    
500     0.744554
501     0.326733
502          0.4
503     0.644554
504     0.869307
Name: EV/GP Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,100.010,N/A,46.95,0.841584,6.596140,0.752475,26.372976,0.877228,11.311629,0.552475,N/A
1,AAL,13.360,N/A,-1.71,0.112871,-60.417952,0.0158416,5.952664,0.0732673,3.098483,0.0574257,N/A
2,AAP,161.440,N/A,29.00,0.623762,3.123759,0.510891,15.086368,0.50099,3.208667,0.0653465,N/A
3,AAPL,466.070,N/A,34.69,0.740594,22.373999,0.940594,25.708773,0.861386,19.428993,0.865347,N/A
4,ABBV,96.680,N/A,21.00,0.427723,-21.463532,0.0257426,12.272585,0.350495,7.672427,0.340594,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,94.320,N/A,28.00,0.6,-3.659682,0.049505,18.841249,0.744554,13.891510,0.744554,N/A
501,ZBH,143.470,N/A,718.10,0.994059,2.390128,0.415842,17.170711,0.572277,7.478460,0.326733,N/A
502,ZBRA,288.222,N/A,31.86,0.693069,8.600669,0.811881,19.480804,0.762376,8.449885,0.4,N/A
503,ZION,35.770,N/A,13.24,0.257426,0.766237,0.132673,18.729176,0.69505,12.206556,0.644554,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,100.010,N/A,46.95,0.841584,6.596140,0.752475,26.372976,0.877228,11.311629,0.552475,0.755941
1,AAL,13.360,N/A,-1.71,0.112871,-60.417952,0.0158416,5.952664,0.0732673,3.098483,0.0574257,0.0648515
2,AAP,161.440,N/A,29.00,0.623762,3.123759,0.510891,15.086368,0.50099,3.208667,0.0653465,0.425248
3,AAPL,466.070,N/A,34.69,0.740594,22.373999,0.940594,25.708773,0.861386,19.428993,0.865347,0.85198
4,ABBV,96.680,N/A,21.00,0.427723,-21.463532,0.0257426,12.272585,0.350495,7.672427,0.340594,0.286139
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,94.320,N/A,28.00,0.6,-3.659682,0.049505,18.841249,0.744554,13.891510,0.744554,0.534653
501,ZBH,143.470,N/A,718.10,0.994059,2.390128,0.415842,17.170711,0.572277,7.478460,0.326733,0.577228
502,ZBRA,288.222,N/A,31.86,0.693069,8.600669,0.811881,19.480804,0.762376,8.449885,0.4,0.666832
503,ZION,35.770,N/A,13.24,0.257426,0.766237,0.132673,18.729176,0.69505,12.206556,0.644554,0.432426


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

Enter the value of your portfolio:1000000


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,HPE,9.830,1994,-293.68,0.0118812,0.768699,0.134653,4.757379,0.0455446,2.575722,0.039604,0.0579208
1,FTI,8.870,2210,-0.68,0.134653,0.526579,0.0732673,2.722313,0.00792079,1.745265,0.0178218,0.0584158
2,AAL,13.360,1467,-1.71,0.112871,-60.417952,0.0158416,5.952664,0.0732673,3.098483,0.0574257,0.0648515
3,CCL,15.610,1256,-3.90,0.10297,0.380999,0.0554455,3.751649,0.0237624,3.458441,0.0811881,0.0658416
4,HFC,25.660,764,-26.18,0.0435644,0.716944,0.120792,3.301222,0.0118812,3.693071,0.0891089,0.0663366
5,HPQ,19.140,1024,9.17,0.172277,-23.323972,0.0237624,5.822424,0.0673267,2.516859,0.0356436,0.0747525
6,XRX,17.800,1101,9.35,0.174257,0.640297,0.0990099,3.620185,0.019802,1.604068,0.0138614,0.0767327
7,TPR,16.010,1224,-20.58,0.049505,1.200922,0.231683,3.870569,0.029703,1.149885,0.0019802,0.0782178
8,LB,27.330,717,-10.49,0.0633663,-5.047485,0.0455446,8.144139,0.170297,3.119623,0.0613861,0.0851485
9,SYF,25.300,775,7.77,0.152475,0.971730,0.180198,2.129599,0.0039604,1.604512,0.0158416,0.0881188


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

## Saving Our Excel Output
As before, saving our Excel output is very easy: